## Tìm kiếm ảnh lỗi

In [1]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm  # Thêm tqdm để hiển thị thanh tiến trình

input_dir = '/kaggle/input/car-cs114/dataset/'
output_csv = '/kaggle/working/ImageErrors.csv'

def find_corrupted_images(input_dir, output_csv):
    error_records = []
    category_mapping = {
        'Others': 0,
        'Honda': 1,
        'Hyundai': 2,
        'KIA': 3,
        'Mazda': 4,
        'Mitsubishi': 5,
        'Suzuki': 6,
        'Toyota': 7,
        'VinFast': 8
    }

    total_files = sum([len(files) for _, _, files in os.walk(input_dir)])

    with tqdm(total=total_files, desc="Đang kiểm tra ảnh", unit="ảnh") as pbar:
        for root, dirs, files in os.walk(input_dir):
            category_name = os.path.basename(root)

            # Bỏ qua các thư mục không có trong category_mapping
            if category_name not in category_mapping:
                pbar.update(len(files))  # Cập nhật tiến trình cho các file bị bỏ qua
                continue

            category_id = category_mapping[category_name]
            for file in files:
                relative_path = os.path.join(category_name, file)
                full_path = os.path.join(root, file)

                try:
                    with Image.open(full_path) as img:
                        img.verify()
                except Exception:
                    error_records.append({"ImageErrorFullPath": relative_path, "CategoryID": category_id})

                pbar.update(1)

    if error_records:
        error_df = pd.DataFrame(error_records)
        error_df.to_csv(output_csv, index=False)
        print(f"Đã tìm thấy {len(error_records)} ảnh bị lỗi. Danh sách được lưu vào {output_csv}")
    else:
        print("Không tìm thấy ảnh bị lỗi nào.")

find_corrupted_images(input_dir, output_csv)


Đang kiểm tra ảnh: 100%|██████████| 38044/38044 [06:16<00:00, 101.13ảnh/s]

Đã tìm thấy 31 ảnh bị lỗi. Danh sách được lưu vào /kaggle/working/ImageErrors.csv


## Check Mode 

In [1]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm  # Thêm tqdm để theo dõi tiến trình

input_dir = '/kaggle/input/car-cs114/dataset/'
output_csv = '/kaggle/working/ImageAttributes.csv'

def check_image_attributes(input_dir, output_csv):
    image_records = []
    category_mapping = {
        'Others': 0,
        'Honda': 1,
        'Hyundai': 2,
        'KIA': 3,
        'Mazda': 4,
        'Mitsubishi': 5,
        'Suzuki': 6,
        'Toyota': 7,
        'VinFast': 8
    }

    # Đếm tổng số file để hiển thị thanh tiến trình
    total_files = sum([len(files) for _, _, files in os.walk(input_dir)])

    # Sử dụng tqdm để theo dõi tiến trình
    with tqdm(total=total_files, desc="Đang kiểm tra ảnh", unit="ảnh") as pbar:
        for root, dirs, files in os.walk(input_dir):
            category_name = os.path.basename(root)

            # Bỏ qua các thư mục không có trong category_mapping
            if category_name not in category_mapping:
                pbar.update(len(files))  # Cập nhật tiến trình cho các file bị bỏ qua
                continue

            category_id = category_mapping[category_name]
            for file in files:
                relative_path = os.path.join(category_name, file)
                full_path = os.path.join(root, file)

                try:
                    with Image.open(full_path) as img:
                        mode = img.mode  # Kiểm tra chế độ ảnh (RGB, RGBA, L, v.v.)
                        has_transparency = (mode == "RGBA" and img.getchannel("A").getextrema()[1] < 255)
                        image_records.append({
                            "ImagePath": relative_path,
                            "CategoryID": category_id,
                            "Mode": mode,
                            "HasTransparency": has_transparency
                        })
                except Exception as e:
                    image_records.append({
                        "ImagePath": relative_path,
                        "CategoryID": category_id,
                        "Mode": "Error",
                        "HasTransparency": "Error"
                    })

                pbar.update(1)  # Cập nhật tiến trình sau khi kiểm tra từng ảnh

    # Lưu kết quả vào file CSV
    if image_records:
        image_df = pd.DataFrame(image_records)
        image_df.to_csv(output_csv, index=False)
        print(f"Đã kiểm tra {len(image_records)} ảnh. Kết quả được lưu vào {output_csv}")
    else:
        print("Không có ảnh nào được kiểm tra.")

check_image_attributes(input_dir, output_csv)

Đang kiểm tra ảnh: 100%|██████████| 38044/38044 [04:10<00:00, 151.82ảnh/s]


Đã kiểm tra 38044 ảnh. Kết quả được lưu vào /kaggle/working/ImageAttributes.csv


In [3]:
import pandas as pd

# Đường dẫn tới file CSV đã tạo
csv_file = '/kaggle/working/ImageAttributes.csv'

def count_image_types(csv_file):
    try:
        # Đọc dữ liệu từ file CSV
        df = pd.read_csv(csv_file)

        # Đếm số loại ảnh theo cột 'Mode'
        mode_counts = df['Mode'].value_counts()

        # Đếm số ảnh có độ trong suốt
        transparency_count = df['HasTransparency'].value_counts()

        # Hiển thị kết quả
        print("Số loại ảnh (Mode):")
        print(mode_counts)
        print("\nSố lượng ảnh có độ trong suốt:")
        print(transparency_count)

    except FileNotFoundError:
        print(f"Không tìm thấy file: {csv_file}")
    except Exception as e:
        print(f"Đã xảy ra lỗi: {e}")

# Gọi hàm
count_image_types(csv_file)

Số loại ảnh (Mode):
Mode
RGB      35236
RGBA      2632
P          140
Error       31
L            3
CMYK         2
Name: count, dtype: int64

Số lượng ảnh có độ trong suốt:
HasTransparency
False    38011
Error       31
True         2
Name: count, dtype: int64


In [ ]:
import cv2

image_path = "/kaggle/input/car-cs114/dataset/Honda/22521560-22521614.Honda.35.jpg"

image = cv2.imread(image_path)

if image is not None:
    cv2.imshow("Image", image)
else:
    print("Không thể đọc ảnh.")
